In [ ]:
import altair as alt
from src.spinorama.load import parse_all_speakers, parse_graphs_speaker, parse_graph_freq
from src.spinorama.contour import compute_contour, compute_contour_smoothed, compute_isoline, reshape
from src.spinorama.graph import graph_freq, graph_contour
import numpy as np

df = parse_all_speakers()

In [ ]:
#parse_graphs_speaker('Emotiva Airmotive 6s')

In [ ]:
for k in df.keys():
  print(k)

In [ ]:
neumann = df['Neumann KH 80']
for k in neumann.keys():
    print(k)

In [ ]:
spinorama_u = neumann['CEA2034_unmelted']
for m in spinorama_u.keys():
    print(m)
spinorama = neumann['CEA2034']
for m in spinorama.keys():
    print(m)

In [ ]:
onaxis = spinorama.loc[spinorama['Measurements']=='On Axis']
print(onaxis)

In [ ]:
from src.spinorama.analysis import estimates

e0, e3, e6, delta = estimates(onaxis)

print('Estimated -3dB point at %1.fHz'% e3)
print('Estimated -6dB point at %1.fHz'% e6)
print('Estimated +/-%1.1fdB from 80hz to 20kz'%delta)


In [ ]:
import math
import locale
from locale import atof
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
import pandas as pd
import numpy as np
from src.spinorama.load import graph_melt

title, df1 = parse_graph_freq('./datas/Neumann KH 80/CEA2034.txt')
print(len(df1.columns))
print(df1.columns)
#print(df1)
#print(df1.reset_index().melt(id_vars='Freq', var_name='Measurements', value_name='dB'))
title2, df2 = parse_graph_freq('./datas/Neumann KH 80/SPL Vertical.txt')
print(len(df2.columns))
print(df2.columns)
print(df2.columns[1])
df2[['Freq', 'On-Axis']].reset_index().melt(id_vars='Freq', var_name='Measurements', value_name='dB')


In [ ]:
from src.spinorama.display import *
# print(df)
speaker = 'Neumann KH 80'
width=400 
heigth=200
spinorama = display_spinorama(df, speaker, width, heigth)
onaxis    = display_onaxis(df, speaker, width, heigth)
inroom    = display_inroom(df, speaker, width, heigth)
ereflex   = display_reflection_early(df, speaker, width, heigth)
hreflex   = display_reflection_horizontal(df, speaker, width, heigth)
vreflex   = display_reflection_vertical(df, speaker, width, heigth)
hspl      = display_spl_horizontal(df, speaker, width, heigth)
vspl      = display_spl_vertical(df, speaker, width, heigth)
hcontour  = display_contour_horizontal(df,speaker, width, heigth)
vcontour  = display_contour_vertical(df,speaker, width, heigth)
hradar    = display_radar_horizontal(df,speaker, width, heigth)
vradar    = display_radar_vertical(df,speaker, width, heigth)


In [ ]:
spinorama

In [ ]:
# display_spinorama(df, speaker, width, heigth)
nearest = alt.selection(
    type='single',
    nearest=True,
    on='mouseover',
    fields=['Freq'],
    empty='none')

dfu = df['Neumann KH 80']['CEA2034']

def display_graph(dfu, width, height):
    # add selectors                                                                                                                          
    selectorsMeasurements = alt.selection_multi(
        fields=['Measurements'], 
        bind='legend')
    scales = alt.selection_interval(
        bind='scales'
    )
    
    # main charts
    line=alt.Chart(dfu).mark_line(
    ).encode(
        alt.X('Freq:Q', scale=alt.Scale(type="log", domain=[20,20000])),
        alt.Y('dB:Q',   scale=alt.Scale(zero=False)),
        alt.Color('Measurements', type='nominal', sort=None),
        opacity=alt.condition(selectorsMeasurements, 
                              alt.value(1), alt.value(0.2))
    ).properties(
        width=width,
        height=height
    )
    
    circle=alt.Chart(dfu).mark_circle(
        size=100
    ).encode(
        alt.X('Freq:Q', scale=alt.Scale(type="log", domain=[20,20000])),
        alt.Y('dB:Q',   scale=alt.Scale(zero=False)),
        alt.Color('Measurements', type='nominal', sort=None),
        opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
        tooltip=['Measurements', 'Freq', 'dB']
    ).transform_calculate(Freq=f'format(datum.Freq, ".0f")',
                          dB=f'format(datum.dB, ".1f")'
    )    

    
    # assemble elements together
    line = (circle+line).add_selection(selectorsMeasurements).add_selection(scales).add_selection(nearest)
    return line

display_graph(dfu, 700, 700)


In [ ]:
# print(df.keys())
speaker1 = 'Neumann KH 80'
speaker2 = 'Kali IN-8'

def augment(dfa, name):
    namearray = [name for i in range(0,len(dfa))]
    dfa['Speaker'] = name
    return dfa

dfu = pd.concat([augment(df[k]['CEA2034'],k) for k in df.keys()])

input_dropdown1 = alt.binding_select(options=[s for s in df.keys()])
selection1 = alt.selection_single(
    fields=['Speaker'], 
    bind=input_dropdown1, 
    name='Select right ', 
    init={'Speaker': speaker1})
input_dropdown2 = alt.binding_select(options=[s for s in df.keys()])
selection2 = alt.selection_single(
    fields=['Speaker'], 
    bind=input_dropdown2, 
    name='Select left ', 
    init={'Speaker': speaker2})

line = alt.Chart(dfu).mark_line().encode(
    alt.X('Freq:Q', scale=alt.Scale(type="log", domain=[20,20000])),
    alt.Y('dB:Q',   scale=alt.Scale(zero=False)),
    alt.Color('Measurements', type='nominal', sort=None)
).properties(
    width=400,
    height=200
)

line1 = line.add_selection(selection1).transform_filter(selection1)
line2 = line.add_selection(selection2).transform_filter(selection2)

#selectors = alt.Chart(df).mark_point().encode(x='Freq:Q', opacity=alt.value(0)).add_selection(nearest)
#points = line.mark_point().encode(opacity=alt.condition(nearest, alt.value(1), alt.value(0)))
#text = line.mark_text(align='left', dx=5, dy=-5).encode(text=alt.condition(nearest, 'dB:Q', alt.value(' ')))
#rules = alt.Chart(df).mark_rule(color='gray').encode(x='Freq:Q').transform_filter(nearest)
line2|line1 #+selectors+points+rules+text

In [ ]:
from scipy import ndimage
import pandas as pd
from astropy.convolution import Gaussian2DKernel

dfu = df['Adam S2V']['SPL Horizontal_unmelted']
x,y,z = compute_contour(dfu)
#print(len(x), len(y), len(z)
def reshape2(x, y, z, nscale):
    nx, ny = x.shape
    # increase the size of the grid by nscale*nscale                                                                                                                                 
    rx, ry = np.meshgrid(np.linspace(np.min(x), np.max(x), nx*nscale),
                         np.linspace(np.min(y), np.max(y), ny*nscale))
    # copy paste the values of z into rz                                                                                                                                             
    rz = np.repeat(np.repeat(z, nscale, axis=0), nscale, axis=1)
    return rx, ry, np.transpose(rz)

rx, ry, rz = reshape2(x, y, z, 2)
#print(len(rx), len(ry), len(rz))

kernel = Gaussian2DKernel(1, mode='oversample', factor=10)
zs = ndimage.convolve(z, kernel.array, mode='mirror')
rzs = ndimage.convolve(rz, kernel.array, mode='mirror')
#print(len(rx), len(ry), len(rzs))
unsmoothed = pd.DataFrame({'x': x.ravel(), 'y': y.ravel(), 'z': z.ravel()})
smoothed = pd.DataFrame({'x': rx.ravel(), 'y': ry.ravel(), 'z': rzs.ravel()})
# value are between -60 and 0
speaker_scale = [-12, -9, -8, -7, -6, -5, -4, -3, -2.5, -2, -1.5, -1, -0.5, 0]
height=12
size=8
s = alt.Chart(smoothed).transform_filter(
            'datum.x>200'
    ).mark_rect(size=size, height=height
    ).encode(alt.X('x:O', scale=alt.Scale(type="log")),
             alt.Y('y:O'),
             alt.Color('z:Q', scale=alt.Scale(domain=speaker_scale))
    ).properties(width=800, height=400)
u = alt.Chart(unsmoothed).transform_filter(
            'datum.x>200'
    ).mark_rect(size=size, height=height
    ).encode(alt.X('x:O', scale=alt.Scale(type="log")),
             alt.Y('y:O'),
             alt.Color('z:Q', scale=alt.Scale(domain=speaker_scale))
    ).properties(width=800, height=400)
alt.vconcat(u,s)